<a href="https://colab.research.google.com/github/hannahweber244/CryptoSentiment/blob/main/CrpytoSentiment_Modell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langdetect
import pandas as pd
import scipy
import random
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
stop_words = stopwords.words('english')
from langdetect import detect
import re, string
#from gensim.models.doc2vec import Doc2Vec
#from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import torch.autograd as autograd         # computation graph
from torch import Tensor 
import torch# tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.nn.functional as F           # layers, activations and more
import torch.optim as optim   
import json

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
!unzip data.json.zip

Archive:  data.json.zip
replace data.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data.json               

In [ ]:
with open("data.json", "r") as f:
    data = json.load(f)

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
descriptions_representation_tfidf = tfidf_vectorizer.fit_transform([" ".join(text) for text, klasse in data.values()])
data_rep = scipy.sparse.csr_matrix.toarray(descriptions_representation_tfidf)
rep_list = list()

for i in range(data_rep.shape[0]):
    
    rep = data_rep[i]
    klasse = data[str(i)][1]
    rep_list.append((rep, klasse))

In [ ]:
random.shuffle(rep_list)
train_ = rep_list[:int(len(rep_list)*0.4)]
test_ = rep_list[int(len(rep_list)*0.4):int(len(rep_list)*0.5)]

In [ ]:
device = torch.device('cpu' if not torch.cuda.is_available() else 'cuda')
def create_batches(liste, batch_size = 64, num_classes = 3):
    
    batches = list()
    batch = list()
    
    targets = list()
    target = list()
    
    for i, (d, t) in enumerate(liste, start = 1):
        if i % batch_size == 0:
            
            label = np.zeros(num_classes)
            t_ = 2 if t == -1 else int(t)
            label[t_] = 1
            label = torch.from_numpy(label).to(device)
            target.append(label)
            target = torch.stack(target)
            targets.append(target)
            
            d = torch.from_numpy(d).to(device)
            batch.append(d)
            
            batch = torch.stack(batch)
            batches.append(batch)
            
            batch = list()
            target = list()
            
        else:
            d = torch.from_numpy(d).to(device)
            batch.append(d)
            
            label = np.zeros(num_classes)
            t_ = 2 if t == -1 else int(t)
            label[t_] = 1
            
            label = torch.from_numpy(label).to(device)
            target.append(label)
    return batches, targets
            


In [ ]:
train_batches, train_targets = create_batches(train_)
test_batches, test_targets = create_batches(test_, batch_size = 1)

In [ ]:
class Sentiment(nn.Module):
    
    def __init__(self, input_dim):
        super(Sentiment, self).__init__()
        
        self.input_layer = nn.Linear(input_dim, 15)
        self.hidden_layer = nn.Linear(15, 7)
        self.output_layer = nn.Linear(7, 3)
        
    def forward(self,x):
        x = F.relu(self.input_layer(x.float()))
        x = F.relu(self.hidden_layer(x))
        x = torch.sigmoid(self.output_layer(x))
        return x
    
def train():
    model.train()
    train_loss = 0
    for batch_id, batch in enumerate(train_batches):
        target = train_targets[batch_id].float()
        
        optimizer.zero_grad()
        out = model(batch).float()
        loss = criterion(out, target).float()
        
        train_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
    return train_loss/len(train_batches)
        
        
def test():
    model.eval()
    correctly_classified = 0
    
    with torch.no_grad():
    
        for batch_id, batch in enumerate(test_batches):
            target = test_targets[batch_id].float()
            
            
            out = model(batch).float()
            
            out = torch.argmax(out)
            tar = torch.argmax(target)
            
            if out == tar:
                correctly_classified += 1
                
    return correctly_classified/len(test_batches)

In [ ]:
loss_df = pd.DataFrame(columns = ['epoche', 'loss', 'accuracy'])
model = Sentiment(115859).float()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.MSELoss()
for e in range(15):
    print(e +1)
    loss_ = train()
    acc = test()
    loss_df.loc[len(loss_df),:] = [e+1, loss_, acc]

In [ ]:
loss_df